In [7]:
import cv2
import numpy as np
import os
from math import tan, atan2, cos, sin, sqrt, pi
from collections import Counter
import pandas as pd
import re

In [8]:
PATH = ['ternausnet/test_data/images/', 'ternausnet/train_data/images/', 'ternausnet/val_data/images/']
#PATH = 'ternausnet/train_data/images/'

result = pd.DataFrame()
true_angles = [0.23, 0.51]

#most_common = []
for path in PATH : 
    for root, _, files in os.walk(path):
        images = [os.path.join(root, f) for f in files]
        for image in images : 
            # Read image 
            image_name = os.path.splitext(os.path.basename(image))[0]
            img = cv2.imread(image, cv2.IMREAD_COLOR) # road.png is the filename
            # Convert the image to gray-scale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Find the edges in the image using canny detector
            edges = cv2.Canny(gray, 110, 140)
            lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=800, maxLineGap=800)
            # Draw lines on the image
            angles = []
            if lines is not None: 
                for line in lines:
                    x1, y1, x2, y2 = line[0]
                    slope = ((y2-y1)/(x1-x2))
                    angles.append(round(slope, 2))
                    #cv2.line(edges, (x1, y1), (x2, y2), (255, 0, 0), 3)
                # Show result
                #print(an220gles)
                c = Counter(angles)
                #c.most_common(1)
                most_commons = c.most_common(3)
                #print ("",most_commons)
                #most_common.append(c.most_common(1)[0][0])
                if most_commons[0][0] in true_angles :
                    result[image_name] = [-int(np.rad2deg(tan(most_commons[0][0])))]
                elif len(most_commons) > 1 : 
                    if most_commons[1][0] in true_angles :
                        result[image_name] = [-int(np.rad2deg(tan(most_commons[1][0])))]
            """
            if lines is not None: 
                for line in lines:
                    x1, y1, x2, y2 = line[0]
                    slope = ((y2-y1)/(x1-x2))
                    if round(slope, 2) == 0.23 or round(slope, 2) == 0.51:
                        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
            #cv2.imshow("Result Image", img)
            #cv2.waitKey(0)
            """
result

,image_31,image_10,image_57,image_107,image_17,image_108,image_1,image_4,image_156,image_115,...,image_15,image_2,image_114,image_34,image_101,image_69,image_113,image_9,image_127,image_154
0,-32,-13,-32,-13,-32,-13,-13,-13,-32,-32,...,-13,-13,-32,-13,-32,-13,-32,-13,-32,-32


In [9]:
def drawAxis(img, p_, q_, color, scale): 
  p = list(p_) 
  q = list(q_) 

  ## [visualization1] 
  angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians 
  hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0])) 
  # Here we lengthen the arrow by a factor of scale 
  q[0] = p[0] - scale * hypotenuse * cos(angle) 
  q[1] = p[1] - scale * hypotenuse * sin(angle) 
  cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA) 
  # create the arrow hooks 
  p[0] = q[0] + 9 * cos(angle + pi / 4) 
  p[1] = q[1] + 9 * sin(angle + pi / 4) 
  cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA) 
  p[0] = q[0] + 9 * cos(angle - pi / 4) 
  p[1] = q[1] + 9 * sin(angle - pi / 4) 
  cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA) 
  ## [visualization1] 

def getOrientation(pts, img): 
  ## [pca] 
  # Construct a buffer used by the pca analysis 
  sz = len(pts) 
  data_pts = np.empty((sz, 2), dtype=np.float64) 
  for i in range(data_pts.shape[0]): 
    data_pts[i,0] = pts[i,0,0] 
    data_pts[i,1] = pts[i,0,1]

  # Perform PCA analysis 
  mean = np.empty((0)) 
  mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean) 
  # Store the center of the object 
  cntr = (int(mean[0,0]), int(mean[0,1])) 
  ## [pca] 
  ## [visualization] 
  # Draw the principal components 
  cv2.circle(img, cntr, 3, (255, 0, 255), 2) 
  p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0]) 
  p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0]) 
  drawAxis(img, cntr, p1, (255, 255, 0), 1) 
  drawAxis(img, cntr, p2, (0, 0, 255), 5) 
  angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians 
  ## [visualization] 
  # Label with the rotation angle 
  label = "  Rotation Angle: " + str(-int(np.rad2deg(angle)) - 90) + " degrees" 
  textbox = cv2.rectangle(img, (cntr[0], cntr[1]-25), (cntr[0] + 250, cntr[1] + 10), (255,255,255), -1) 
  cv2.putText(img, label, (cntr[0], cntr[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA) 
  return angle 

In [11]:
# Load the image 
PATH = ['ternausnet/test_data/masks_with_axons/', 'ternausnet/train_data/masks_with_axons/', 'ternausnet/val_data/masks_with_axons/']
masks = pd.DataFrame()
for path in PATH : 
    for root, _, files in os.walk(path):
        images = [os.path.join(root, f) for f in files]
        for image in images : 
            angles = []
            image_name = os.path.splitext(os.path.basename(image))[0]
            image_name = re.sub('mask', 'image', image_name)
            img = cv2.imread(image) 
            # Was the image there? 
            #cv2.imshow('Input Image', img) 
            # Convert image to grayscale 
            #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
            # Convert image to binary 
            #_, bw = cv2.threshold(img, 0, 240, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
            #cv2.imshow('Gray Image', bw) 
            th, threshed = cv2.threshold(img, 240, 256, cv2.THRESH_BINARY_INV)
            cv2.imshow('Gray Image', threshed) 
            #contours = cv2.getStructuringElement(shape=cv2.MORPH_ELLIPSE, ksize=(5, 5))
            #closing = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, contours, iterations=1)
            threshed = cv2.cvtColor(threshed, cv2.COLOR_BGR2GRAY)
            contours, _ = cv2.findContours(threshed, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            for i,c in enumerate(contours):
              # area of each contour
              area = cv2.contourArea(c)
              # ignore contour which is too small or large
              if area < 1e2 or 1e5 < area:
                continue

              # draw each contour only for visualization
              cv2.drawContours(threshed, contours, i, (0, 0, 255), 2)
              # find orientation of each shape
              angle = getOrientation(c,threshed)
              angles.append(-int(np.rad2deg(angle)) - 90)
            masks[image_name] = [angles]
            #cv2.imshow('oe',threshed)
            #cv2.imshow('Output Image', closing) 
            #cv2.waitKey(0) 
            #cv2.destroyAllWindows() 
masks

<ipython-input-11-6445259d56e3>:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  masks[image_name] = [angles]


,image_99,image_55,image_108,image_111,image_123,image_115,image_78,image_61,image_57,image_60,...,image_146,image_88,image_135,image_132,image_154,image_101,image_121,image_9,image_69,image_145
0,"[-111, -143, -64]",[-208],"[-155, -135, -82, -198]",[-111],[-83],"[-109, -81, -200]","[-88, -148, -149]","[-121, -179, -135]",[],"[-176, -116, -83, -74, -87, -57]",...,[],[-213],[],[],"[-58, -156, -202, -165, -88, -114, -192, -123]",[-151],"[-70, -104, -183]",[-127],"[-103, -95]",[-77]
